# The Vanishing/Exploding Gradients Problems
 * The vanishing gradients problem occurs when the gradients of the loss function with respect to the parameters become extremely small during backpropagation.
 * Conversely, the exploding gradients problem arises when the gradients of the loss function grow exponentially as they propagate backward through the layers during backpropagation.


## Weight Initialization
 * Proper initialization of network weights can help alleviate both vanishing and exploding gradients problems. 
 * Techniques like __Xavier__ initialization and __He__ initialization are commonly used to ensure that the weights are initialized in a way that keeps the signal propagated through the network.

In [3]:
# Importing essential package
import tensorflow as tf
from tensorflow import keras

# List comprehension to get the names of initializers in Keras without underscores
initializer_names = [name for name in dir(keras.initializers) if not name.startswith("_")]

# Printing the initializer names
print(initializer_names)

['Constant', 'GlorotNormal', 'GlorotUniform', 'HeNormal', 'HeUniform', 'Identity', 'Initializer', 'LecunNormal', 'LecunUniform', 'Ones', 'Orthogonal', 'RandomNormal', 'RandomUniform', 'TruncatedNormal', 'VarianceScaling', 'Zeros', 'constant', 'deserialize', 'get', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform', 'identity', 'lecun_normal', 'lecun_uniform', 'ones', 'orthogonal', 'random_normal', 'random_uniform', 'serialize', 'truncated_normal', 'variance_scaling', 'zeros']
